In [2]:
import pandas as pd
import sys
import numpy as np

In [3]:
# set parameter
net_name = ["Lenet", "GoogLenet", "Squeezenet", "Alexnet", "Mobilenet"]
device_name = "RedmiNote9P"
fre_list = ["576000", "768000", "1017600", "1248000", "1324800", "1516800", "1612800", "1708800", "1804800"]  # Set Frequency in KHz:  没有办法降频到300000
freq_big_list=["787200", "979200", "1036800", "1248000", "1401600", "1555200", "1766400", "1900800", "2073600", "2131200", "2208000"]
# batchsizes = [1, 2, 4, 8, 16]
batchsizes = [1, 2, 3, 4, 5, 6, 7, 8]

In [5]:
# 测试不同numCore的energy
device_name = "MEIZU_numCore_16"
df = pd.DataFrame(columns=net_name, index=batchsizes)

path_to_baseline = "./RedmiNote9P_1804800/processed_data/Lenet/processed_data_Lenet.csv"
perform = pd.read_csv(path_to_baseline, index_col=0)
baseline_warmup = 10
baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值
print("baseline")
print(baseline)
#baseline = 0
for net in net_name:
    # print(net)
    for batchsize in batchsizes:
        df[net][batchsize] = 0
        path_to_stamp = "./" + device_name + "/train_stamp/" + net + "/" + "train_stamp_" + str(batchsize) + ".result" 
        data = pd.read_table(path_to_stamp, header=None, delimiter="\t")
        for line in data[0]:
            if "Begin training" in line:
                trainStart = int(line.split()[3].split("N")[0])
            if "End trainning" in line:
                trainEnd = int(line.split()[3].split("N")[0])
            if "Begin inferring" in line:
                inferStart = int(line.split()[3].split("N")[0])
            if "End inferring" in line:
                inferEnd = int(line.split()[3].split("N")[0])
        path_to_perform = "./" + device_name + "/processed_data/" + net + "/processed_data_" + net + ".csv"
        perform = pd.read_csv(path_to_perform, index_col=0)
        # baseline_warmup = 5
        # baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值
        P = []
        for index in perform.index: # 按行对数据进行遍历
            if int(perform.loc[index]["stamp"]) > trainStart and int(perform.loc[index]["stamp"]) < trainEnd:
                print(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"])
                P.append(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"]-baseline)
        avg_P = np.mean(P)
        df[net][batchsize] = avg_P * (trainEnd - trainStart)/10**15
df.to_csv("./numCore_energy_MEIZU.csv")
# print(df)
                

1607380
3281197257600
4417459787880
4170495430200
4074185802240
3120888850200
5309233078050
2927302739370
5167926954900
5436766722960
6085191512280
5389224444540
5246836529760
5295837975000
4931539358760
3775993001550
5206454122080
4681290422400
4513515397050
5977019483820
4946163717600
3575375125560
5182744281000
4547738493000
5293796930520
4214426152230
4823563213770
3007349248410
5117214489030
5463693605760
3788779795290
2297536647390
4805598224490
2511922383180
3991107780450
2923601728320
5704224616560
4675465278300
4830787885560
3766087989450
2590467741630
2838559769130
3529831944720
3646074946230
4543190148930
5560509959220
3063102616530
5987907241830
4365115627050
3339653660040
2408092538550
3078261366180
4785201958680
2348214476160
5167708149000
2341964359980
3270189030300
4319779584960
3264022600020
3180782932140
3428936784000
5699297344110
5128596489630
5337845999520
5344687424100
4717824022800
3141880011090
3286847469600
2768398292700
3464853566310
2850580596240
499630840191

In [16]:
# 单个freq，不同网络、不同batchsize 训练过程中消耗的能量 p*t p使用所有训练时刻的电压*电流均值（只看usb）减去基准功率（前5条数据的最小值/均值/最大值）t使用end - start ｜stamp
device_name = "MEIZU_big"
df = pd.DataFrame(columns=net_name, index=batchsizes)

path_to_baseline = "./MEIZU_big/processed_data/Lenet/processed_data_Lenet.csv"
perform = pd.read_csv(path_to_baseline, index_col=0)
baseline_warmup = 5
baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值

for net in net_name:
    print(net)
    for batchsize in batchsizes:
        df[net][batchsize] = 0
        path_to_stamp = "./" + device_name + "/train_stamp/" + net + "/" + "train_stamp_" + str(batchsize) + ".result" 
        data = pd.read_table(path_to_stamp, header=None, delimiter="\t")
        for line in data[0]:
            if "Begin training" in line:
                trainStart = int(line.split()[3].split("N")[0])
            if "End trainning" in line:
                trainEnd = int(line.split()[3].split("N")[0])
            if "Begin inferring" in line:
                inferStart = int(line.split()[3].split("N")[0])
            if "End inferring" in line:
                inferEnd = int(line.split()[3].split("N")[0])
        path_to_perform = "./" + device_name + "/processed_data/" + net + "/processed_data_" + net + ".csv"
        perform = pd.read_csv(path_to_perform, index_col=0)
        #print(perform)
        # baseline_warmup = 5
        # baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值
        P = []
        for index in perform.index: # 按行对数据进行遍历
            if int(perform.loc[index]["stamp"]) > trainStart and int(perform.loc[index]["stamp"]) < trainEnd:
                # print(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"])
                P.append(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"]-baseline)
        avg_P = np.mean(P)
        print(avg_P)
        df[net][batchsize] = avg_P * (trainEnd - trainStart)/10**15
df.to_csv("./batchsize_energy_MEIZU_small.csv")
# print(df)
                

Lenet
897632967497.1428
921430549071.4286
996959087256.6666
1222844264895.0
1205607834843.158
GoogLenet
1285078179965.4546
1393646363479.138
1450791334616.1904
1580727442220.1135
1538754093744.5833
Squeezenet
1451033395440.4917
1568467450745.2703
1572913602042.1875
1657986595919.1892
1635869539970.5715
Alexnet
1686383367109.608
1345752704256.9092
1471482069145.532
1494079731189.4443
1565721796874.0789
Mobilenet
nan
nan
nan
nan
nan


In [1]:
# 测单个sample的energy
device_name = "RedmiNote9P_1804800"
df = pd.DataFrame(columns=net_name, index=batchsizes)

data = pd.read_csv("./batchsize_energy.csv", index_col=0)

# int warmUp = 2;
# int measureIterations = 32/BatchSize + 5;
# int loops = 1;
# if (NetName == "Lenet") { measureIterations = 1024/BatchSize + 100; } 
for batch in batchsizes:
    for net in net_name:
        if net == "Lenet":
            batchnum = 2 + 1024/batch + 100 + 1
        elif net == "Mobilenet":
            batchnum = 26
        else:
            batchnum = 2 + 32/batch + 5 + 1
        df[net][batch] = data[net][batch]/(batchnum*batch)

df.to_csv("./singleSample_energy.csv")

NameError: name 'pd' is not defined

In [7]:
# 测单个batch的energy
device_name = "RedmiNote9P_1804800"
df = pd.DataFrame(columns=net_name, index=batchsizes)

data = pd.read_csv("./batchsize_energy.csv", index_col=0)

# int warmUp = 2;
# int measureIterations = 32/BatchSize + 5;
# int loops = 1;
# if (NetName == "Lenet") { measureIterations = 1024/BatchSize + 100; } 
for batch in batchsizes:
    for net in net_name:
        if net == "Lenet":
            batchnum = 2 + 1024/batch + 100 + 1
        elif net == "Mobilenet":
            batchnum = 26
        else:
            batchnum = 2 + 32/batch + 5 + 1
        df[net][batch] = data[net][batch]/(batchnum)

df.to_csv("./singleBatch_energy.csv")

In [1]:
# 多个freq，不同网络、相同batchsize 训练过程中消耗的能量 p*t p使用所有训练时刻的电压*电流均值（只看usb）减去基准功率（前5条数据的最小值/均值/最大值）t使用end - start ｜stamp
device_name = "RedmiNote9P_big"

# num = 6
df = pd.DataFrame(columns=net_name, index=freq_big_list[:])
print(net_name)
batchsize = 8

path_to_baseline = "./RedmiNote9P_1804800/processed_data/GoogLenet/processed_data_GoogLenet.csv"
perform = pd.read_csv(path_to_baseline, index_col=0)
baseline_warmup = 5
baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值

for net in net_name:
    print(net)
    for freq in freq_big_list[:]:
        df[net][freq] = 0
        path_to_stamp = "./" + device_name + "_" + freq + "/train_stamp/" + net + "/" + "train_stamp_" + str(batchsize) + ".result" 
        data = pd.read_table(path_to_stamp, header=None, delimiter="\t")
        for line in data[0]:
            if "Begin training" in line:
                trainStart = int(line.split()[3].split("N")[0])
            if "End trainning" in line:
                trainEnd = int(line.split()[3].split("N")[0])
            if "Begin inferring" in line:
                inferStart = int(line.split()[3].split("N")[0])
            if "End inferring" in line:
                inferEnd = int(line.split()[3].split("N")[0])
        path_to_perform = "./" + device_name + "_" + freq + "/processed_data/" + net + "/processed_data_" + net + ".csv"
        perform = pd.read_csv(path_to_perform, index_col=0)
        # baseline_warmup = 5
        # baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值
        # print("baseline:" + str(baseline))
        P = []
        for index in perform.index: # 按行对数据进行遍历
            if int(perform.loc[index]["stamp"]) > trainStart and int(perform.loc[index]["stamp"]) < trainEnd:
                # print(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"])
                P.append(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"]-baseline)
        avg_P = np.mean(P)
        df[net][freq] = avg_P * (trainEnd - trainStart)/10**15
df.to_csv("./freq_energy_big_" + str(batchsize) + ".csv")
# print(df)
                

NameError: name 'pd' is not defined

In [47]:
# 多个freq，不同网络、相同batchsize 训练过程中单个batchsize花费的时间
df = pd.DataFrame(columns=net_name, index=freq_big_list[:])
print(net_name)
batchsize = 8

for net in net_name:
    print(net)
    for freq in freq_big_list[:]:
        df[net][freq] = 0
        path_to_time = "./" + device_name + "_" + freq + "/processed_data/" + net + "/" + "time_table_" + net + ".csv" 
        data = pd.read_csv(path_to_time, index_col=0)
        df[net][freq] = data["expr_train"][data["batchsize"]==batchsize].values[0]
        print(df[net][freq])
df.to_csv("./freq_time_big_" + str(batchsize) + ".csv")

['Lenet', 'Squeezenet', 'GoogLenet', 'Alexnet', 'Mobilenet']
Lenet
28.582015999999996
24.539742999999998
22.615423
19.772318
21.04421
19.767084
18.616203
17.88176
17.382812
17.22415
16.821335
Squeezenet
3649.518311
3282.081055
2978.6826170000004
2632.847168
2558.954834
2372.65625
2268.185547
2222.007568
2160.753418
2151.734375
2133.762939
GoogLenet
2440.8208010000003
2145.172852
1931.1707760000002
1706.99292
1645.251343
1517.885742
1436.457275
1384.4373779999999
1354.236938
1335.089966
1322.5233150000001
Alexnet
3926.209717
3368.363525
3209.965332
2746.877197
2610.552734
2391.094727
2235.107178
2149.0205079999996
2088.334229
2097.108643
2042.672485
Mobilenet
5112.344727000001
4699.855957
4277.132324
3993.370361
3910.695312
3719.395996
3609.209473
3556.8957520000004
3517.828857
3500.000488
3485.2780759999996


In [6]:
# 测试大小核之间的差距
device_name = "MEIZU"

kernel = ["big", "small"]
# num = 6
df = pd.DataFrame(columns=net_name, index=kernel[:])
print(net_name)
batchsize = 8

path_to_baseline = "./RedmiNote9P_big/processed_data/GoogLenet/processed_data_GoogLenet.csv"
perform = pd.read_csv(path_to_baseline, index_col=0)
baseline_warmup = 5
baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值

for net in net_name:
    print(net)
    for freq in freq_big_list[:]:
        df[net][freq] = 0
        path_to_stamp = "./" + device_name + "_" + freq + "/train_stamp/" + net + "/" + "train_stamp_" + str(batchsize) + ".result" 
        data = pd.read_table(path_to_stamp, header=None, delimiter="\t")
        for line in data[0]:
            if "Begin training" in line:
                trainStart = int(line.split()[3].split("N")[0])
            if "End trainning" in line:
                trainEnd = int(line.split()[3].split("N")[0])
            if "Begin inferring" in line:
                inferStart = int(line.split()[3].split("N")[0])
            if "End inferring" in line:
                inferEnd = int(line.split()[3].split("N")[0])
        path_to_perform = "./" + device_name + "_" + freq + "/processed_data/" + net + "/processed_data_" + net + ".csv"
        perform = pd.read_csv(path_to_perform, index_col=0)
        # baseline_warmup = 5
        # baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值
        # print("baseline:" + str(baseline))
        P = []
        for index in perform.index: # 按行对数据进行遍历
            if int(perform.loc[index]["stamp"]) > trainStart and int(perform.loc[index]["stamp"]) < trainEnd:
                # print(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"])
                P.append(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"]-baseline)
        avg_P = np.mean(P)
        df[net][freq] = avg_P * (trainEnd - trainStart)/10**15
df.to_csv("./freq_energy_big_" + str(batchsize) + ".csv")
# print(df)
                

['Lenet', 'GoogLenet', 'Squeezenet', 'Alexnet', 'Mobilenet']
Lenet


FileNotFoundError: [Errno 2] File ./MEIZU_787200/train_stamp/Lenet/train_stamp_8.result does not exist: './MEIZU_787200/train_stamp/Lenet/train_stamp_8.result'

In [13]:
# 多种kernel，多种频率，batchsum=25
device_name = "MEIZU"
# kernel_list = ["1_h", "10_h", "3_h", "30_h", "f_h", "f0_h","ff_h",
#                 "1_m", "10_m", "3_m", "30_m", "f_m", "f0_h","ff_m",
#                 "1_l", "10_l", "3_l", "30_l", "f_l", "f0_l","ff_l"]

kernel_list = ["10_h", "3_h", "30_h", "f0_h", "ff_h"]
# num = 6
df = pd.DataFrame(columns=net_name, index=kernel_list[:])
print(net_name)
batchsize = 16

baseline = int(100000 * 5000000)# 日常功率的基准值

for net in net_name[3:4]:
    print(net)
    for freq in kernel_list[:]:
        df[net][freq] = 0
        path_to_stamp = "./" + device_name + "_" + freq + "/train_stamp/" + net + "/" + "train_stamp_" + str(batchsize) + ".result" 
        data = pd.read_table(path_to_stamp, header=None, delimiter="\t")
        for line in data[0]:
            if "Begin training" in line:
                trainStart = int(line.split()[3].split("N")[0])
            if "End trainning" in line:
                trainEnd = int(line.split()[3].split("N")[0])
            if "Begin inferring" in line:
                inferStart = int(line.split()[3].split("N")[0])
            if "End inferring" in line:
                inferEnd = int(line.split()[3].split("N")[0])
        path_to_perform = "./" + device_name + "_" + freq + "/processed_data/" + net + "/processed_data_" + net + ".csv"
        print(path_to_perform)
        perform = pd.read_csv(path_to_perform, index_col=0)
        #print(perform)
        # baseline_warmup = 5
        # baseline = int(np.mean(perform.loc[:baseline_warmup]["usb_current"])) * int(np.mean(perform.loc[:baseline_warmup]["usb_voltage"]))# 日常功率的基准值
        # print("baseline:" + str(baseline))
        P = []
        for index in perform.index: # 按行对数据进行遍历
            if int(perform.loc[index]["stamp"]) > trainStart and int(perform.loc[index]["stamp"]) < trainEnd:
                # print(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"])
                P.append(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"]-baseline)
                #print(perform.loc[index]["usb_current"]*perform.loc[index]["usb_voltage"])
        avg_P = np.mean(P)
        df[net][freq] = avg_P * (trainEnd - trainStart)/10**15
        df[net][freq] = df[net][freq]/25
        print(df[net][freq])
df.to_csv("./freq_kernel_MEIZU_" + str(batchsize) + ".csv")
# print(df)
                

['Lenet', 'GoogLenet', 'Squeezenet', 'Alexnet', 'Mobilenet']
Alexnet
./MEIZU_10_h/processed_data/Alexnet/processed_data_Alexnet.csv
10.17025268892782
./MEIZU_3_h/processed_data/Alexnet/processed_data_Alexnet.csv
13.824160943920496
./MEIZU_30_h/processed_data/Alexnet/processed_data_Alexnet.csv
8.632528228630187
./MEIZU_f0_h/processed_data/Alexnet/processed_data_Alexnet.csv
6.933880568671724
./MEIZU_ff_h/processed_data/Alexnet/processed_data_Alexnet.csv
12.965262586838028


In [ ]:
device_name = "MEIZU"
# kernel_list = ["1_h", "10_h", "3_h", "30_h", "f_h", "f0_h","ff_h",
#                 "1_m", "10_m", "3_m", "30_m", "f_m", "f0_h","ff_m",
#                 "1_l", "10_l", "3_l", "30_l", "f_l", "f0_l","ff_l"]

kernel_list = ["10_h", "3_h", "30_h", "f0_h", "ff_h"]
df = pd.DataFrame(columns=net_name, index=kernel_list[:])
print(net_name)
batchsize = 8

for net in net_name[3:4]:
    print(net)
    for freq in kernel_list[:]:
        df[net][freq] = 0
        path_to_time = "./" + device_name + "_" + freq + "/processed_data/" + net + "/" + "time_table_" + net + ".csv" 
        data = pd.read_csv(path_to_time, index_col=0)
        df[net][freq] = data["expr_train"][data["batchsize"]==batchsize].values[0]
        print(df[net][freq])
df.to_csv("./freq_kernel_MEIZU_" + str(batchsize) + ".csv")